# **Machine Learning II: Bayesian & Unsupervised Methods Project**
**Ana-Maria Borduselu**

## Libraries

In [3]:
# Core Libraries
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

# Dimensionality Reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# UMAP (optional – requires pip install umap-learn)
import umap

# Clustering
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram

# Anomaly Detection
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor


# Utility / Settings
import warnings

## Data Wrangling

In [10]:
# Load Hepatitis C dataset
df = pd.read_csv("HepatitisCdata.csv", index_col=0)

df.reset_index(drop=True, inplace=True)
df.index = range(1, len(df) + 1)

# view the first 5 rows
print(df.head())

        Category  Age Sex   ALB   ALP   ALT   AST   BIL    CHE  CHOL   CREA  \
1  0=Blood Donor   32   m  38.5  52.5   7.7  22.1   7.5   6.93  3.23  106.0   
2  0=Blood Donor   32   m  38.5  70.3  18.0  24.7   3.9  11.17  4.80   74.0   
3  0=Blood Donor   32   m  46.9  74.7  36.2  52.6   6.1   8.84  5.20   86.0   
4  0=Blood Donor   32   m  43.2  52.0  30.6  22.6  18.9   7.33  4.74   80.0   
5  0=Blood Donor   32   m  39.2  74.1  32.6  24.8   9.6   9.15  4.32   76.0   

    GGT  PROT  
1  12.1  69.0  
2  15.6  76.5  
3  33.2  79.3  
4  33.8  75.7  
5  29.9  68.7  


#### 2.1 Data cleaning

* Verify variable types (continuous vs categorical)
* Handle missing values:

  * Remove records with excessive missingness
  * Impute remaining missing values using median (robust to skewness)
* Remove obvious data entry errors (negative or impossible values)


In [16]:
# -----------------------------
# 2.1 DATA CLEANING
# -----------------------------

# (A) Verify variable types (categorical vs numerical)
# Here we assume:
# - 'Sex' and 'Category' are categorical
# - Everything else (Age + lab markers) should be numeric
# If your dataset uses different names (e.g. "Sex" as "Sex", "Category" as "Category"),
# adjust the column names below accordingly.
categorical_cols = ["Sex", "Category"]
numeric_cols = [c for c in df.columns if c not in categorical_cols]

# Ensure numeric columns are numeric (convert if they were read as strings)
# Any non-convertible values become NaN (missing), which we will handle later.
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# (B) Handle missing values
# Step 1: Remove records with excessive missingness
# - "excessive" is a design choice. Common thresholds: 20%–40%.
# Here we use 30% missing across columns as an example.
row_missing_fraction = df.isna().mean(axis=1)
df = df.loc[row_missing_fraction <= 0.30].copy()

# Step 2: Impute remaining missing values (median is robust to skewness)
# - We'll impute numeric columns with median.
# - For Sex (categorical), we'll use most frequent.
num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

df[numeric_cols] = num_imputer.fit_transform(df[numeric_cols])
df[["Sex"]] = cat_imputer.fit_transform(df[["Sex"]])

# (C) Remove obvious data entry errors (negative or impossible values)
# Examples:
# - Age should be > 0
# - lab values should generally be >= 0 (some can’t be negative physiologically)
# We'll enforce non-negativity for lab markers and a reasonable age range.
# Adjust these rules if your dataset includes valid edge cases.
df = df[(df["Age"] > 0) & (df["Age"] < 120)].copy()

lab_markers = ["ALB","ALP","ALT","AST","BIL","CHE","CHOL","CREA","GGT","PROT"]
# Keep only lab markers that actually exist in your file (prevents key errors)
lab_markers = [c for c in lab_markers if c in df.columns]

# Remove rows where any lab marker is negative (impossible biologically)
for c in lab_markers:
    df = df[df[c] >= 0].copy()



        Category  Age Sex   ALB   ALP   ALT   AST   BIL    CHE  CHOL   CREA  \
1  0=Blood Donor   32   m  38.5  52.5   7.7  22.1   7.5   6.93  3.23  106.0   
2  0=Blood Donor   32   m  38.5  70.3  18.0  24.7   3.9  11.17  4.80   74.0   
3  0=Blood Donor   32   m  46.9  74.7  36.2  52.6   6.1   8.84  5.20   86.0   
4  0=Blood Donor   32   m  43.2  52.0  30.6  22.6  18.9   7.33  4.74   80.0   
5  0=Blood Donor   32   m  39.2  74.1  32.6  24.8   9.6   9.15  4.32   76.0   

    GGT  PROT  
1  12.1  69.0  
2  15.6  76.5  
3  33.2  79.3  
4  33.8  75.7  
5  29.9  68.7  


#### 2.2 Distributional analysis

* Inspect univariate distributions
* Identify skewed variables (ALT, AST, GGT, BIL)
* Assess outliers (initially, not remove them)

In [ ]:
# -----------------------------
# 2.2 DISTRIBUTIONAL ANALYSIS
# -----------------------------
# At this stage, we DO NOT remove outliers yet.
# We only identify skewness and potential heavy tails to justify transformations later.

# Identify skewness of each numeric variable
skewness = df[numeric_cols].skew(numeric_only=True).sort_values(ascending=False)
print("\nSkewness (descending):\n", skewness)

# Define known typically skewed variables in this dataset
skewed_vars = [c for c in ["ALT", "AST", "GGT", "BIL"] if c in df.columns]
print("\nSkewed variables to consider for log transform:", skewed_vars)

# Optional: basic quantiles to see outliers (still not removing them)
print("\nQuantiles for skewed vars:")
print(df[skewed_vars].quantile([0.5, 0.75, 0.9, 0.95, 0.99]))



#### 2.3 Scaling and transformation

* Apply **log transformation** to heavily skewed variables
* Standardize all continuous variables (z-score scaling)
* Encode sex as binary for inclusion in numerical analysis

📌 *Rationale:* Distance-based methods require comparable feature scales.

In [ ]:
# -----------------------------
# 2.3 SCALING AND TRANSFORMATION
# -----------------------------
# Rationale: distance-based algorithms (PCA, K-means, etc.) are sensitive to scale.
# We will:
# 1) encode Sex into numeric
# 2) log-transform heavily skewed variables
# 3) standardize (z-score) all continuous variables

# (A) Encode Sex as binary
# The dataset often uses 'm'/'f' or 'M'/'F'.
# We'll map common variants. Adjust if your dataset encodes sex differently.
sex_map = {"m": 1, "M": 1, "male": 1, "Male": 1,
            "f": 0, "F": 0, "female": 0, "Female": 0}
df["Sex_bin"] = df["Sex"].map(sex_map)

# If some values didn't map (NaN), fill with the mode as a safe default.
df["Sex_bin"] = df["Sex_bin"].fillna(df["Sex_bin"].mode()[0]).astype(int)

# (B) Separate Category for post hoc interpretation ONLY
# This is the label-like column we do NOT use to train unsupervised models.
y_category = df["Category"].copy()

# (C) Create the feature matrix X (exclude Category and original Sex)
# Keep Sex_bin instead.
feature_cols = ["Age", "Sex_bin"] + lab_markers
X = df[feature_cols].copy()

# (D) Log-transform heavily skewed variables
# Use log1p(x) = log(1 + x), safe when x can be 0.
for c in skewed_vars:
    if c in X.columns:
        X[c] = np.log1p(X[c])

# (E) Standardize all features (mean=0, std=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Optional: put back into a DataFrame for readability
X_scaled = pd.DataFrame(X_scaled, columns=feature_cols, index=df.index)

print("\nFinal feature matrix ready for unsupervised learning:")
print("X_scaled shape:", X_scaled.shape)
print("Category kept separately (post hoc only). Unique categories:", sorted(y_category.unique()))

# ============================================================
# At this point:
# - X_scaled is ready for PCA, clustering, anomaly detection
# - y_category is saved only to interpret clusters afterwards
# ============================================================


## **Exploratory Data Analysis (EDA)**

### Objectives

* Understand baseline variability
* Identify correlations and redundancy
* Detect early structure or anomalies

### Methods

* Summary statistics per variable
* Correlation matrix and heatmap
* Pairwise plots (selected variables)

### Outputs

* Identification of strongly correlated markers (ALT–AST, ALB–PROT)
* Justification for dimensionality reduction

In [ ]:
# ------------------------------------------------------------
# ASSUMPTION / INPUTS FROM PREVIOUS STEP
# ------------------------------------------------------------
# You should already have:
# - df: cleaned dataframe (still contains original variables)
# - X: feature dataframe before scaling/transforms (Age, Sex_bin, lab markers, log transforms applied if you chose)
# - X_scaled: standardized features (DataFrame)
# - y_category: stored separately for post hoc interpretation
#
# If you ran Part A, you have:
#   df, X, X_scaled, y_category
# ------------------------------------------------------------


# ============================================================
# 3) EXPLORATORY DATA ANALYSIS (EDA)
# ============================================================
# Objectives:
# 1) Understand baseline variability (ranges, distributions)
# 2) Identify correlations and redundancy (highly correlated markers)
# 3) Detect early structure or anomalies (extreme values, clusters visually)


In [ ]:
# -----------------------------
# 3.1 Summary statistics
# -----------------------------
# Summary stats help you understand:
# - central tendency (mean/median)
# - spread (std/IQR)
# - extreme values (min/max)
# This is crucial for medical lab data because many markers are skewed.

print("\n--- Summary statistics (features only) ---")
print(X.describe().T)

# Optional: more robust statistics (median, IQR) for skewed variables
print("\n--- Robust stats (median + IQR) ---")
median = X.median(numeric_only=True)
q1 = X.quantile(0.25, numeric_only=True)
q3 = X.quantile(0.75, numeric_only=True)
iqr = q3 - q1
robust_stats = pd.DataFrame({"median": median, "Q1": q1, "Q3": q3, "IQR": iqr})
print(robust_stats)

In [ ]:
# -----------------------------
# 3.2 Correlation matrix (redundancy detection)
# -----------------------------
# Correlations reveal redundancy:
# - Example: ALT and AST often move together (both reflect liver injury)
# - ALB and PROT can correlate (albumin is a major component of total protein)
#
# We compute correlations on X (interpretable). Correlation is scale-invariant.
# If you want, you can also compute on X_scaled.

corr = X.corr(numeric_only=True)

print("\n--- Top absolute correlations (excluding self-correlations) ---")
# Extract upper triangle correlations only (avoid duplicates)
corr_abs = corr.abs()
upper = corr_abs.where(np.triu(np.ones(corr_abs.shape), k=1).astype(bool))
top_pairs = upper.stack().sort_values(ascending=False).head(15)
print(top_pairs)


In [ ]:
# -----------------------------
# 3.3 Heatmap of correlations
# -----------------------------
# A heatmap is a fast way to see blocks of correlated features.
# If seaborn isn't available, you can skip this section.

plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=False, cmap="coolwarm", center=0)
plt.title("Correlation Matrix (Features)")
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------
# 3.4 Pairwise plots (selected variables)
# -----------------------------
# Pairwise plots help detect:
# - obvious groupings (early cluster structure)
# - outliers (extreme values)
# - non-linear relationships
#
# Since full pairplots can be heavy, select a small set of clinically meaningful markers.
# Recommended set: ALT, AST, GGT, BIL, ALB, PROT (+ Age)
selected = [c for c in ["Age", "ALB", "PROT", "ALT", "AST", "BIL", "GGT"] if c in X.columns]

print("\nPairplot variables used:", selected)

if len(selected) >= 2:
    # Pairplot can take time on large datasets, but this dataset is usually manageable.
    # We avoid using Category here to respect the unsupervised philosophy.
    # (You may color by Category later ONLY for post hoc interpretation.)
    sns.pairplot(X[selected], corner=True, plot_kws={"s": 10, "alpha": 0.6})
    plt.suptitle("Pairwise Plots (Selected Biomarkers)", y=1.02)
    plt.show()
else:
    # Minimal alternative: scatterplots for a few key relationships
    # ALT vs AST (expected strong correlation)
    if "ALT" in X.columns and "AST" in X.columns:
        plt.figure()
        plt.scatter(X["ALT"], X["AST"], s=10, alpha=0.6)
        plt.xlabel("ALT")
        plt.ylabel("AST")
        plt.title("ALT vs AST (Expected strong correlation)")
        plt.tight_layout()
        plt.show()

    # ALB vs PROT (potential redundancy)
    if "ALB" in X.columns and "PROT" in X.columns:
        plt.figure()
        plt.scatter(X["ALB"], X["PROT"], s=10, alpha=0.6)
        plt.xlabel("ALB")
        plt.ylabel("PROT")
        plt.title("ALB vs PROT (Potential redundancy)")
        plt.tight_layout()
        plt.show()


In [ ]:
# -----------------------------
# 3.5 Early anomaly detection (EDA-level)
# -----------------------------
# At EDA stage, we do NOT run full anomaly algorithms yet.
# We just flag extreme values using quantiles to see whether the dataset
# contains heavy tails / extreme cases that may influence clustering.

high_quantile = 0.99  # you can also check 0.995 or 0.999
extreme_flags = {}

for c in selected:
    if c in X.columns and pd.api.types.is_numeric_dtype(X[c]):
        threshold = X[c].quantile(high_quantile)
        extreme_flags[c] = int((X[c] > threshold).sum())

print(f"\n--- Number of points above the {high_quantile:.2f} quantile (selected vars) ---")
for k, v in extreme_flags.items():
    print(f"{k}: {v}")


In [ ]:

# ============================================================
# 3.6 Expected EDA Outputs (to write in your report)
# ============================================================
# From the top correlations, you will usually observe:
# - ALT–AST strong positive correlation (both liver injury enzymes)
# - ALB–PROT positive correlation (albumin contributes to total protein)
#
# These findings justify dimensionality reduction because:
# - correlated variables increase redundancy in feature space
# - PCA can compress correlated markers into fewer latent components
#
# You can copy the printed "top_pairs" into your report narrative.
# ============================================================


## **Dimensionality Reduction and Latent Structure Discovery**

In [ ]:
# ============================================================
# Hepatitis C Unsupervised Learning Project
# Part C — Dimensionality Reduction & Latent Structure Discovery
#   4.1 PCA (for modeling + interpretability)
#   4.2 UMAP / t-SNE (visualization only)
#   (Corrected: Category handling for matplotlib coloring)
# ============================================================


# ------------------------------------------------------------
# ASSUMPTION / INPUTS FROM PREVIOUS STEPS
# ------------------------------------------------------------
# You should already have:
# - X_scaled: standardized feature matrix (DataFrame)
# - y_category: stored separately for post hoc interpretation (Series)
# ------------------------------------------------------------


In [ ]:
# ============================================================
# 0) FIX CATEGORY FOR PLOTTING
# ============================================================
# Your Category may be stored as strings like "0=Blood Donor".
# Matplotlib scatter(c=...) requires numeric values.
# We keep BOTH:
# - y_category_label: original labels (for legends / tables)
# - y_category_num: numeric encoding (for colormap coloring)

y_category_label = y_category.astype(str)

# Extract leading digit(s) if Category looks like "0=Blood Donor"
# If Category is already numeric, this still works.
y_category_num = (
    y_category_label
    .str.extract(r"^(\d+)")[0]
    .fillna(y_category_label)         # fallback if it doesn't start with digits
)

# Convert to numeric safely
y_category_num = pd.to_numeric(y_category_num, errors="coerce")

# If any values still failed conversion, map unique labels to integers
if y_category_num.isna().any():
    label_map = {lab: i for i, lab in enumerate(sorted(y_category_label.unique()))}
    y_category_num = y_category_label.map(label_map).astype(int)
else:
    y_category_num = y_category_num.astype(int)

print("Category labels example:", y_category_label.unique()[:5])
print("Category numeric codes example:", sorted(y_category_num.unique()))



### 4.1 Principal Component Analysis (PCA)

**Purpose**

* Reduce dimensionality
* Identify dominant axes of variation
* Improve clustering stability

**Procedure**

* Apply PCA on standardized data
* Examine explained variance ratio
* Retain components explaining ~70–80% variance

**Interpretation**

* Analyze PCA loadings to link components to physiological processes (e.g. inflammation vs synthetic function)

In [ ]:
# ============================================================
# 4.1 PRINCIPAL COMPONENT ANALYSIS (PCA)
# ============================================================

# -----------------------------
# 4.1.1 Fit PCA on standardized data
# -----------------------------
pca_full = PCA(n_components=None, random_state=42)
X_pca_full = pca_full.fit_transform(X_scaled)

explained = pca_full.explained_variance_ratio_
cum_explained = np.cumsum(explained)

print("\n--- PCA explained variance ratio (first 10 PCs) ---")
for i, v in enumerate(explained[:10], start=1):
    print(f"PC{i}: {v:.4f} (cumulative: {cum_explained[i-1]:.4f})")


In [ ]:
# -----------------------------
# 4.1.2 Plot cumulative explained variance
# -----------------------------
plt.figure(figsize=(7, 4))
plt.plot(range(1, len(cum_explained) + 1), cum_explained, marker="o")
plt.axhline(0.70, linestyle="--")
plt.axhline(0.80, linestyle="--")
plt.xlabel("Number of Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.title("PCA — Cumulative Explained Variance")
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------
# 4.1.3 Choose number of components (~70–80% variance)
# -----------------------------
target_variance = 0.80
k = int(np.argmax(cum_explained >= target_variance) + 1)
print(f"\nChosen number of components k = {k} to reach ≥ {target_variance*100:.0f}% variance.")

pca = PCA(n_components=k, random_state=42)
X_pca = pca.fit_transform(X_scaled)

pc_cols = [f"PC{i}" for i in range(1, k + 1)]
X_pca_df = pd.DataFrame(X_pca, columns=pc_cols, index=X_scaled.index)

print("\nX_pca_df shape:", X_pca_df.shape)

In [ ]:
# -----------------------------
# 4.1.4 PCA loadings (interpretation)
# -----------------------------
loadings = pd.DataFrame(
    pca.components_.T,
    index=X_scaled.columns,
    columns=pc_cols
)

n_top = 5
pcs_to_show = min(3, k)

print("\n--- PCA Loadings Interpretation (Top contributors) ---")
for pc in pc_cols[:pcs_to_show]:
    top_pos = loadings[pc].sort_values(ascending=False).head(n_top)
    top_neg = loadings[pc].sort_values(ascending=True).head(n_top)
    print(f"\n{pc} — Top + contributors:\n{top_pos}")
    print(f"{pc} — Top - contributors:\n{top_neg}")

# Optional: barplot for PC1 loadings
pc_to_plot = "PC1"
plt.figure(figsize=(8, 4))
plt.bar(loadings.index, loadings[pc_to_plot].values)
plt.xticks(rotation=90)
plt.ylabel("Loading weight")
plt.title(f"PCA Loadings — {pc_to_plot}")
plt.tight_layout()
plt.show()



### 4.2 Non-linear embeddings (Visualization only)

**Methods**

* UMAP
* t-SNE (optional comparison)

**Purpose**

* Visualize local and global structure
* Identify gradients, overlap, and density variations

📌 *Important:* These embeddings are **not used for clustering**, only for interpretation.

In [ ]:
# ============================================================
# 4.2 NON-LINEAR EMBEDDINGS (VISUALIZATION ONLY)
# ============================================================

# -----------------------------
# 4.2.1 t-SNE (optional comparison)
# -----------------------------
tsne = TSNE(
    n_components=2,
    perplexity=30,
    learning_rate="auto",
    init="pca",
    random_state=42
)
X_tsne = tsne.fit_transform(X_scaled)

plt.figure(figsize=(6, 5))
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], s=10, alpha=0.6)
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.title("t-SNE Embedding (Visualization Only)")
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------
# 4.2.2 UMAP (recommended visualization)
# -----------------------------
reducer = umap.UMAP(
        n_components=2,
        n_neighbors=15,
        min_dist=0.1,
        random_state=42)

X_umap = reducer.fit_transform(X_scaled)
plt.figure(figsize=(6, 5))
plt.scatter(X_umap[:, 0], X_umap[:, 1], s=10, alpha=0.6)
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.title("UMAP Embedding (Visualization Only)")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# OPTIONAL: POST HOC COLORING BY CATEGORY (INTERPRETATION ONLY)
# ============================================================

def scatter_by_category_numeric(X_2d, title, y_num):
    """2D scatter colored by numeric category codes (safe for matplotlib)."""
    plt.figure(figsize=(6, 5))
    scatter = plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y_num, s=10, alpha=0.7)
    plt.xlabel("Dim 1")
    plt.ylabel("Dim 2")
    plt.title(title + " (colored by Category — post hoc)")
    plt.colorbar(scatter, label="Category code")
    plt.tight_layout()
    plt.show()

def scatter_by_category_labels(X_2d, title, y_lab):
    """2D scatter with a legend using the original string labels."""
    labs = pd.Series(y_lab).astype(str)
    plt.figure(figsize=(6, 5))
    for lab in sorted(labs.unique()):
        mask = labs == lab
        plt.scatter(X_2d[mask, 0], X_2d[mask, 1], s=10, alpha=0.7, label=lab)
    plt.xlabel("Dim 1")
    plt.ylabel("Dim 2")
    plt.title(title + " (post hoc labels)")
    plt.legend(markerscale=2, bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

# Use numeric coloring (fast + safe)
scatter_by_category_numeric(X_tsne, "t-SNE Embedding", y_category_num)

if X_umap is not None:
    scatter_by_category_numeric(X_umap, "UMAP Embedding", y_category_num)

# If you prefer readable legends (slower but nicer for reports), use:
# scatter_by_category_labels(X_tsne, "t-SNE Embedding", y_category_label)
# if X_umap is not None:
#     scatter_by_category_labels(X_umap, "UMAP Embedding", y_category_label)



In [ ]:
# ============================================================
# OUTPUTS FOR NEXT STEPS
# ============================================================
# - X_pca_df : PCA-reduced representation (use for clustering stability)
# - loadings : interpret PCs (inflammation vs synthetic function)
# - X_tsne / X_umap : visualization only
# ============================================================


## **Patient Stratification via Clustering**

In [ ]:
# ============================================================
# Hepatitis C Unsupervised Learning Project
# Part D — Patient Stratification via Clustering
#   5.1 K-Means + Elbow + Silhouette
#   5.2 Gaussian Mixture Models (GMM) + BIC + soft membership
#   5.3 Hierarchical clustering + dendrogram
# ============================================================


# ------------------------------------------------------------
# ASSUMPTION / INPUTS FROM PREVIOUS STEPS
# ------------------------------------------------------------
# You should already have:
# - X_pca_df : PCA-reduced representation (DataFrame)
# - (optional) y_category, y_category_num for later post hoc interpretation
# ------------------------------------------------------------

# ------------------------------------------------------------
# We cluster on PCA-reduced data for:
# - noise reduction
# - improved stability
# - reduced dimensionality
# ------------------------------------------------------------
X_cluster = X_pca_df.copy()

### 5.1 K-Means Clustering

**Purpose**

* Establish a baseline clustering solution

**Procedure**

* Apply K-means on PCA-reduced data
* Select number of clusters using:

  * Elbow method
  * Silhouette score

**Output**

* Cluster centroids
* Cluster sizes
* Silhouette values


In [ ]:
# ============================================================
# 5.1 K-MEANS CLUSTERING
# ============================================================

# -----------------------------
# 5.1.1 Elbow Method
# -----------------------------
# Purpose:
# - Examine inertia (within-cluster sum of squares)
# - Identify diminishing returns when increasing k

inertias = []
k_range = range(2, 11)  # try 2 to 10 clusters

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=20)
    kmeans.fit(X_cluster)
    inertias.append(kmeans.inertia_)

plt.figure()
plt.plot(list(k_range), inertias, marker="o")
plt.xlabel("Number of clusters (k)")
plt.ylabel("Inertia (Within-cluster SSE)")
plt.title("Elbow Method for K-Means")
plt.tight_layout()
plt.show()

In [ ]:
# -----------------------------
# 5.1.2 Silhouette Score
# -----------------------------
# Purpose:
# - Measure cluster separation and cohesion
# - Range: [-1, 1]
# - Higher = better defined clusters

silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=20)
    labels = kmeans.fit_predict(X_cluster)
    score = silhouette_score(X_cluster, labels)
    silhouette_scores.append(score)

plt.figure()
plt.plot(list(k_range), silhouette_scores, marker="o")
plt.xlabel("Number of clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score vs k")
plt.tight_layout()
plt.show()

# Choose optimal k based on silhouette score (and sanity-check with elbow)
optimal_k = int(list(k_range)[int(np.argmax(silhouette_scores))])
print(f"Selected k = {optimal_k} based on silhouette score.")


In [ ]:
# -----------------------------
# 5.1.3 Final K-Means model
# -----------------------------
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20)
cluster_labels_km = kmeans_final.fit_predict(X_cluster)

# Store cluster assignments
X_cluster["Cluster_KMeans"] = cluster_labels_km

In [ ]:
# -----------------------------
# Outputs (K-Means)
# -----------------------------

# Cluster sizes
cluster_sizes = X_cluster["Cluster_KMeans"].value_counts().sort_index()
print("\nCluster sizes (K-Means):")
print(cluster_sizes)

# Cluster centroids in PCA space
centroids_km = pd.DataFrame(
    kmeans_final.cluster_centers_,
    columns=X_cluster.columns.drop("Cluster_KMeans")
)
print("\nCluster centroids (K-Means, PCA space):")
print(centroids_km)

final_sil = silhouette_score(X_cluster.drop(columns=["Cluster_KMeans"]), cluster_labels_km)
print("\nFinal silhouette score (K-Means):", final_sil)

# Optional: visualize clusters in PC1/PC2
plt.figure(figsize=(6, 5))
plt.scatter(X_cluster["PC1"], X_cluster["PC2"], c=X_cluster["Cluster_KMeans"], s=10, alpha=0.7)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("K-Means Clusters in PCA Space")
plt.colorbar(label="Cluster")
plt.tight_layout()
plt.show()



### 5.2 Gaussian Mixture Models (GMM)

**Purpose**

* Model overlapping clusters probabilistically

**Procedure**

* Fit GMM with varying number of components
* Select model using BIC
* Analyze soft cluster memberships

📌 *Rationale:* Reflects biological overlap between disease stages.


In [ ]:
# ============================================================
# 5.2 GAUSSIAN MIXTURE MODELS (GMM)
# ============================================================

# -----------------------------
# 5.2.1 Select number of components using BIC
# -----------------------------
# Purpose:
# - Choose the number of mixture components that best balances
#   fit and complexity (lower BIC is better).

bic_scores = []
gmm_models = []

X_gmm = X_cluster.drop(columns=["Cluster_KMeans"])

for k in k_range:
    gmm = GaussianMixture(n_components=k, random_state=42)
    gmm.fit(X_gmm)
    bic_scores.append(gmm.bic(X_gmm))
    gmm_models.append(gmm)

plt.figure()
plt.plot(list(k_range), bic_scores, marker="o")
plt.xlabel("Number of components")
plt.ylabel("BIC (lower is better)")
plt.title("GMM Model Selection (BIC)")
plt.tight_layout()
plt.show()

optimal_gmm_k = int(list(k_range)[int(np.argmin(bic_scores))])
print(f"Selected GMM components = {optimal_gmm_k}")


In [ ]:
# -----------------------------
# 5.2.2 Fit final GMM
# -----------------------------
gmm_final = GaussianMixture(n_components=optimal_gmm_k, random_state=42)
gmm_final.fit(X_gmm)

cluster_labels_gmm = gmm_final.predict(X_gmm)
cluster_probs = gmm_final.predict_proba(X_gmm)

X_cluster["Cluster_GMM"] = cluster_labels_gmm

# -----------------------------
# Outputs (GMM)
# -----------------------------

print("\nFirst 5 soft cluster probability vectors (GMM):")
print(pd.DataFrame(cluster_probs[:5], columns=[f"GMM_{i}" for i in range(optimal_gmm_k)]))

print("\nCluster sizes (GMM):")
print(pd.Series(cluster_labels_gmm).value_counts().sort_index())

# Optional: visualize GMM clusters in PC1/PC2
plt.figure(figsize=(6, 5))
plt.scatter(X_cluster["PC1"], X_cluster["PC2"], c=X_cluster["Cluster_GMM"], s=10, alpha=0.7)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("GMM Clusters in PCA Space")
plt.colorbar(label="Cluster")
plt.tight_layout()
plt.show()

### 5.3 Hierarchical Clustering

**Purpose**

* Explore multi-scale structure
* Visualize progression patterns

**Procedure**

* Apply agglomerative clustering
* Generate dendrogram
* Compare different cut heights

In [ ]:
# ============================================================
# 5.3 HIERARCHICAL CLUSTERING
# ============================================================

# -----------------------------
# 5.3.1 Generate linkage matrix + dendrogram
# -----------------------------
# Purpose:
# - Explore multi-scale structure
# - Visualize potential progression patterns
#
# IMPORTANT: Use ONLY PCA features (exclude cluster label columns)
X_hc = X_cluster.drop(columns=["Cluster_KMeans", "Cluster_GMM"])

Z = linkage(X_hc, method="ward")

plt.figure(figsize=(10, 5))
dendrogram(Z, truncate_mode="level", p=5)
plt.title("Hierarchical Clustering Dendrogram (Truncated)")
plt.xlabel("Samples (truncated view)")
plt.ylabel("Distance")
plt.tight_layout()
plt.show()

In [ ]:
# -----------------------------
# 5.3.2 Agglomerative clustering (Ward)
# -----------------------------
# Choose n_clusters (we use the same as K-Means for comparison)
agg = AgglomerativeClustering(n_clusters=optimal_k, linkage="ward")
cluster_labels_hc = agg.fit_predict(X_hc)

X_cluster["Cluster_Hierarchical"] = cluster_labels_hc

print("\nCluster sizes (Hierarchical):")
print(pd.Series(cluster_labels_hc).value_counts().sort_index())

# Optional: visualize hierarchical clusters in PC1/PC2
plt.figure(figsize=(6, 5))
plt.scatter(X_cluster["PC1"], X_cluster["PC2"], c=X_cluster["Cluster_Hierarchical"], s=10, alpha=0.7)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Hierarchical Clusters in PCA Space")
plt.colorbar(label="Cluster")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# OUTPUTS FOR NEXT STEPS
# ============================================================
# - X_cluster now contains:
#   * PCA components (PC1..PCk)
#   * Cluster_KMeans
#   * Cluster_GMM
#   * Cluster_Hierarchical
#
# Use X_cluster + y_category (post hoc) in Section 6 (continuum analysis)
# ============================================================


## **Disease Progression and Latent Continuum Analysis**

In [ ]:
# ============================================================
# 6) DISEASE PROGRESSION & LATENT CONTINUUM ANALYSIS
# ============================================================

# ------------------------------------------------------------
# ASSUMPTION
# ------------------------------------------------------------
# You already have:
# - X_pca_df  → PCA representation (DataFrame)
# - X_cluster → dataframe containing cluster labels (DataFrame)
# - y_category → original diagnostic categories (post hoc only; may be strings like "0=Blood Donor")
# - loadings → PCA loadings (DataFrame)
# ------------------------------------------------------------

# ============================================================
# 0) FIX CATEGORY TYPE FOR PLOTTING (IMPORTANT)
# ============================================================
# Matplotlib scatter(c=...) needs numeric values, but Category might be strings.
# We'll create both numeric and label versions:
y_category_label = pd.Series(y_category).astype(str)

# Try to extract a leading integer from labels like "0=Blood Donor"
y_category_num = y_category_label.str.extract(r"^(\d+)")[0]
y_category_num = pd.to_numeric(y_category_num, errors="coerce")

# If extraction fails (still NaNs), map unique labels to integer codes
if y_category_num.isna().any():
    label_map = {lab: i for i, lab in enumerate(sorted(y_category_label.unique()))}
    y_category_num = y_category_label.map(label_map).astype(int)
else:
    y_category_num = y_category_num.astype(int)

# ------------------------------------------------------------
# For clarity, merge everything into one analysis dataframe
# ------------------------------------------------------------
analysis_df = X_pca_df.copy()

analysis_df["Cluster_KMeans"] = X_cluster["Cluster_KMeans"].values
analysis_df["Cluster_GMM"] = X_cluster["Cluster_GMM"].values
analysis_df["Cluster_Hierarchical"] = X_cluster["Cluster_Hierarchical"].values

analysis_df["Category_label"] = y_category_label.values
analysis_df["Category_num"] = y_category_num.values


### Objectives

* Determine whether disease structure is discrete or continuous
* Understand cluster ordering

### Methods

* Project clusters onto PCA components
* Examine cluster centroids along principal axes
* Overlay post hoc diagnostic categories

### Interpretation

* Identify monotonic trends along severity axes
* Assess overlap between adjacent groups

In [ ]:

# ============================================================
# 6.1 PROJECT CLUSTERS ONTO PCA SPACE
# ============================================================

plt.figure(figsize=(6, 5))
plt.scatter(
    analysis_df["PC1"],
    analysis_df["PC2"],
    c=analysis_df["Cluster_KMeans"],
    s=10,
    alpha=0.7
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("K-Means Clusters in PCA Space")
plt.colorbar(label="Cluster (K-Means)")
plt.tight_layout()
plt.show()

# Optional: Compare GMM clusters in same space
plt.figure(figsize=(6, 5))
plt.scatter(
    analysis_df["PC1"],
    analysis_df["PC2"],
    c=analysis_df["Cluster_GMM"],
    s=10,
    alpha=0.7
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("GMM Clusters in PCA Space")
plt.colorbar(label="Cluster (GMM)")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 6.2 EXAMINE CLUSTER CENTROIDS ALONG PRINCIPAL AXES
# ============================================================

# Compute centroids in PCA space (you can extend to PC3, PC4 if needed)
centroids_pca = analysis_df.groupby("Cluster_KMeans")[["PC1", "PC2"]].mean()

print("\nCluster centroids (K-Means, PCA space):")
print(centroids_pca)

# Plot centroids for PC1 (often a candidate severity axis)
plt.figure()
plt.bar(centroids_pca.index.astype(str), centroids_pca["PC1"])
plt.xlabel("Cluster (K-Means)")
plt.ylabel("Mean PC1")
plt.title("Cluster Ordering Along PC1 (K-Means)")
plt.tight_layout()
plt.show()

# OPTIONAL: Show ordering explicitly
centroid_order = centroids_pca["PC1"].sort_values().index.tolist()
print("\nClusters ordered by increasing mean PC1 (K-Means):", centroid_order)

In [ ]:
# ============================================================
# 6.3 LINK PCA AXES TO PHYSIOLOGY
# ============================================================

# Loadings interpretation helper: show strongest contributors for a PC
def show_top_loadings(loadings_df, pc="PC1", n=7):
    print(f"\nTop + loadings for {pc}:")
    print(loadings_df[pc].sort_values(ascending=False).head(n))
    print(f"\nTop - loadings for {pc}:")
    print(loadings_df[pc].sort_values(ascending=True).head(n))

show_top_loadings(loadings, "PC1", n=5)
show_top_loadings(loadings, "PC2", n=5)

# Tip for report interpretation:
# - PC dominated by ALT/AST/GGT/BIL tends to reflect "injury/cholestasis/inflammation"
# - PC dominated by ALB/CHE/PROT/CHOL tends to reflect "synthetic/metabolic function"
# (Direction sign is arbitrary: + vs - does not change interpretation.)


In [ ]:
# ============================================================
# 6.4 OVERLAY POST HOC DIAGNOSTIC CATEGORIES
# ============================================================

plt.figure(figsize=(6, 5))
sc = plt.scatter(
    analysis_df["PC1"],
    analysis_df["PC2"],
    c=analysis_df["Category_num"],   # numeric codes for safe coloring
    s=10,
    alpha=0.7
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA Projection Colored by Diagnostic Category (Post Hoc)")
plt.colorbar(sc, label="Category (numeric code)")
plt.tight_layout()
plt.show()

# OPTIONAL: If you prefer readable legend with labels (can be crowded):
def scatter_with_label_legend(df2d, title):
    plt.figure(figsize=(6, 5))
    for lab in sorted(df2d["Category_label"].unique()):
        mask = df2d["Category_label"] == lab
        plt.scatter(df2d.loc[mask, "PC1"], df2d.loc[mask, "PC2"], s=10, alpha=0.7, label=lab)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title(title)
    plt.legend(markerscale=2, bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

# Uncomment if you want:
# scatter_with_label_legend(analysis_df, "PCA Projection with Category Labels (Post Hoc)")


In [ ]:
# ============================================================
# 6.5 CATEGORY DISTRIBUTION PER CLUSTER
# ============================================================

# Use numeric categories for stable ordering in tables
cluster_category_table = pd.crosstab(
    analysis_df["Cluster_KMeans"],
    analysis_df["Category_num"],
    normalize="index"
)

print("\nCategory distribution within each K-Means cluster (row-normalized):")
print(cluster_category_table)

plt.figure(figsize=(8, 4))
plt.imshow(cluster_category_table.values, aspect="auto")
plt.xticks(range(len(cluster_category_table.columns)), cluster_category_table.columns)
plt.yticks(range(len(cluster_category_table.index)), cluster_category_table.index)
plt.xlabel("Category (numeric)")
plt.ylabel("Cluster (K-Means)")
plt.title("Cluster vs Category Distribution (Proportions)")
plt.colorbar(label="Proportion")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 6.6 MONOTONIC TREND ANALYSIS
# ============================================================

# Compute mean PC1 per diagnostic category (numeric)
category_means = analysis_df.groupby("Category_num")["PC1"].mean().sort_index()

print("\nMean PC1 per Category (numeric):")
print(category_means)

plt.figure()
plt.plot(category_means.index, category_means.values, marker="o")
plt.xlabel("Diagnostic Category (numeric)")
plt.ylabel("Mean PC1")
plt.title("Category Trend Along PC1 (Monotonicity Check)")
plt.tight_layout()
plt.show()

# OPTIONAL: also check PC2 trend
category_means_pc2 = analysis_df.groupby("Category_num")["PC2"].mean().sort_index()
plt.figure()
plt.plot(category_means_pc2.index, category_means_pc2.values, marker="o")
plt.xlabel("Diagnostic Category (numeric)")
plt.ylabel("Mean PC2")
plt.title("Category Trend Along PC2")
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 6.7 OVERLAP ASSESSMENT (RANGES + BOX-PLOT)
# ============================================================

# Print PC1 min/max range by category to quantify overlap
for cat in sorted(analysis_df["Category_num"].unique()):
    subset = analysis_df[analysis_df["Category_num"] == cat]
    print(f"\nCategory {cat} — PC1 range:")
    print(f"Min: {subset['PC1'].min():.2f}, Max: {subset['PC1'].max():.2f}, "
          f"IQR: {(subset['PC1'].quantile(0.75) - subset['PC1'].quantile(0.25)):.2f}")

# Boxplot gives an immediate visual of overlap between adjacent categories
cats_sorted = sorted(analysis_df["Category_num"].unique())
data_for_box = [analysis_df.loc[analysis_df["Category_num"] == c, "PC1"].values for c in cats_sorted]

plt.figure(figsize=(8, 4))
plt.boxplot(data_for_box, labels=[str(c) for c in cats_sorted], showfliers=False)
plt.xlabel("Diagnostic Category (numeric)")
plt.ylabel("PC1")
plt.title("PC1 Distribution by Category (Overlap Assessment)")
plt.tight_layout()
plt.show()

# ============================================================
# OPTIONAL: "SEVERITY SCORE" IN LATENT SPACE (USEFUL FOR CONCLUSION)
# ============================================================
# A simple unsupervised severity proxy:
# - Use PC1 as a severity score if it shows monotonic trend with Category.
analysis_df["Severity_proxy_PC1"] = analysis_df["PC1"]

# Examine how clusters align with severity proxy
severity_by_cluster = analysis_df.groupby("Cluster_KMeans")["Severity_proxy_PC1"].mean().sort_values()
print("\nMean severity proxy (PC1) by K-Means cluster (sorted):")
print(severity_by_cluster)


## **Anomaly Detection and Atypical Profiles**

### 7.1 Statistical Outlier Detection

**Methods**

* Z-score
* IQR method

**Purpose**

* Identify global deviations from population norms

### 7.2 Model-based Anomaly Detection

**Methods**

* Isolation Forest
* Local Outlier Factor (LOF)

**Purpose**

* Detect local and structural anomalies
* Identify patients poorly represented by clusters


### Outputs

* Anomaly scores per patient
* Overlap between anomalies and advanced disease categories
* Identification of potential data quality issues


In [ ]:
# ============================================================
# 7) ANOMALY DETECTION & ATYPICAL PROFILES (FIXED)
# ============================================================

# ------------------------------------------------------------
# ASSUMPTION
# ------------------------------------------------------------
# Available:
# - X_scaled  → standardized feature matrix (DataFrame)
# - analysis_df → contains PCA columns (PC1, PC2, ...) + cluster labels
# - y_category → original Category series (strings like "0=Blood Donor" OR numeric)
# ------------------------------------------------------------

# Ensure X_scaled is a DataFrame with correct index
if isinstance(X_scaled, np.ndarray):
    X_scaled = pd.DataFrame(X_scaled, index=analysis_df.index)

# ============================================================
# 0) ENSURE CATEGORY COLUMNS EXIST IN analysis_df (NO KEYERROR)
# ============================================================

# If analysis_df already has Category_label/Category_num from Section 6, keep them.
# Otherwise rebuild them from y_category.
if ("Category_label" not in analysis_df.columns) or ("Category_num" not in analysis_df.columns):
    y_category_label = pd.Series(y_category, index=analysis_df.index).astype(str)

    # Extract leading digits from labels like "0=Blood Donor"
    y_category_num = pd.to_numeric(
        y_category_label.str.extract(r"^(\d+)")[0],
        errors="coerce"
    )

    # If extraction fails for some entries, map labels to codes
    if y_category_num.isna().any():
        label_map = {lab: i for i, lab in enumerate(sorted(y_category_label.unique()))}
        y_category_num = y_category_label.map(label_map).astype(int)
    else:
        y_category_num = y_category_num.astype(int)

    analysis_df["Category_label"] = y_category_label.values
    analysis_df["Category_num"] = y_category_num.values



In [ ]:
# ============================================================
# 7.1 STATISTICAL OUTLIER DETECTION
# ============================================================

# --- Z-score method (global outliers) ---
z_scores = np.abs(zscore(X_scaled, nan_policy="omit"))
z_outliers = (z_scores > 3).any(axis=1)

analysis_df["Z_Outlier"] = z_outliers.astype(int)
print("\nNumber of Z-score outliers:", int(z_outliers.sum()))

# --- IQR method (robust global outliers) ---
Q1 = X_scaled.quantile(0.25)
Q3 = X_scaled.quantile(0.75)
IQR = Q3 - Q1

iqr_outliers = ((X_scaled < (Q1 - 1.5 * IQR)) |
                (X_scaled > (Q3 + 1.5 * IQR))).any(axis=1)

analysis_df["IQR_Outlier"] = iqr_outliers.astype(int)
print("Number of IQR outliers:", int(iqr_outliers.sum()))



In [ ]:
# ============================================================
# 7.2 MODEL-BASED ANOMALY DETECTION
# ============================================================

# --- Isolation Forest (structural/global) ---
iso = IsolationForest(contamination=0.05, random_state=42)
iso.fit(X_scaled)

analysis_df["Isolation_Score"] = iso.decision_function(X_scaled)   # higher = normal
analysis_df["Isolation_Outlier"] = (iso.predict(X_scaled) == -1).astype(int)

print("\nIsolation Forest detected anomalies:", int(analysis_df["Isolation_Outlier"].sum()))

# --- LOF (local density anomalies) ---
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
lof_pred = lof.fit_predict(X_scaled)

analysis_df["LOF_Score"] = lof.negative_outlier_factor_
analysis_df["LOF_Outlier"] = (lof_pred == -1).astype(int)

print("LOF detected anomalies:", int(analysis_df["LOF_Outlier"].sum()))


In [ ]:
# ============================================================
# 7.3 VISUALIZE ANOMALIES IN PCA SPACE
# ============================================================

plt.figure(figsize=(6, 5))
sc = plt.scatter(analysis_df["PC1"], analysis_df["PC2"],
                 c=analysis_df["Isolation_Outlier"], s=10, alpha=0.7)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Isolation Forest Outliers in PCA Space")
plt.colorbar(sc, label="Outlier (1=Yes)")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 7.4 OVERLAP WITH DIAGNOSTIC CATEGORIES (POST HOC)
# ============================================================

iso_category_table = pd.crosstab(
    analysis_df["Isolation_Outlier"],
    analysis_df["Category_num"],
    normalize="columns"
)
print("\nIsolation Forest Outliers by Category (proportion within category):")
print(iso_category_table)

lof_category_table = pd.crosstab(
    analysis_df["LOF_Outlier"],
    analysis_df["Category_num"],
    normalize="columns"
)
print("\nLOF Outliers by Category (proportion within category):")
print(lof_category_table)


In [ ]:
# ============================================================
# 6) DISEASE PROGRESSION & LATENT CONTINUUM ANALYSIS
# ============================================================

# ------------------------------------------------------------
# ASSUMPTION
# ------------------------------------------------------------
# You already have:
# - X_pca_df  → PCA representation (DataFrame)
# - X_cluster → dataframe containing cluster labels (DataFrame)
# - y_category → original diagnostic categories (post hoc only; may be strings like "0=Blood Donor")
# - loadings → PCA loadings (DataFrame)
# ------------------------------------------------------------

# ============================================================
# 0) FIX CATEGORY TYPE FOR PLOTTING (IMPORTANT)
# ============================================================
# Matplotlib scatter(c=...) needs numeric values, but Category might be strings.
# We'll create both numeric and label versions:
y_category_label = pd.Series(y_category).astype(str)

# Try to extract a leading integer from labels like "0=Blood Donor"
y_category_num = y_category_label.str.extract(r"^(\d+)")[0]
y_category_num = pd.to_numeric(y_category_num, errors="coerce")

# If extraction fails (still NaNs), map unique labels to integer codes
if y_category_num.isna().any():
    label_map = {lab: i for i, lab in enumerate(sorted(y_category_label.unique()))}
    y_category_num = y_category_label.map(label_map).astype(int)
else:
    y_category_num = y_category_num.astype(int)

# ------------------------------------------------------------
# For clarity, merge everything into one analysis dataframe
# ------------------------------------------------------------
analysis_df = X_pca_df.copy()

analysis_df["Cluster_KMeans"] = X_cluster["Cluster_KMeans"].values
analysis_df["Cluster_GMM"] = X_cluster["Cluster_GMM"].values
analysis_df["Cluster_Hierarchical"] = X_cluster["Cluster_Hierarchical"].values

analysis_df["Category_label"] = y_category_label.values
analysis_df["Category_num"] = y_category_num.values


### Objectives

* Determine whether disease structure is discrete or continuous
* Understand cluster ordering

### Methods

* Project clusters onto PCA components
* Examine cluster centroids along principal axes
* Overlay post hoc diagnostic categories

### Interpretation

* Identify monotonic trends along severity axes
* Assess overlap between adjacent groups

In [ ]:

# ============================================================
# 6.1 PROJECT CLUSTERS ONTO PCA SPACE
# ============================================================

plt.figure(figsize=(6, 5))
plt.scatter(
    analysis_df["PC1"],
    analysis_df["PC2"],
    c=analysis_df["Cluster_KMeans"],
    s=10,
    alpha=0.7
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("K-Means Clusters in PCA Space")
plt.colorbar(label="Cluster (K-Means)")
plt.tight_layout()
plt.show()

# Optional: Compare GMM clusters in same space
plt.figure(figsize=(6, 5))
plt.scatter(
    analysis_df["PC1"],
    analysis_df["PC2"],
    c=analysis_df["Cluster_GMM"],
    s=10,
    alpha=0.7
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("GMM Clusters in PCA Space")
plt.colorbar(label="Cluster (GMM)")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 6.2 EXAMINE CLUSTER CENTROIDS ALONG PRINCIPAL AXES
# ============================================================

# Compute centroids in PCA space (you can extend to PC3, PC4 if needed)
centroids_pca = analysis_df.groupby("Cluster_KMeans")[["PC1", "PC2"]].mean()

print("\nCluster centroids (K-Means, PCA space):")
print(centroids_pca)

# Plot centroids for PC1 (often a candidate severity axis)
plt.figure()
plt.bar(centroids_pca.index.astype(str), centroids_pca["PC1"])
plt.xlabel("Cluster (K-Means)")
plt.ylabel("Mean PC1")
plt.title("Cluster Ordering Along PC1 (K-Means)")
plt.tight_layout()
plt.show()

# OPTIONAL: Show ordering explicitly
centroid_order = centroids_pca["PC1"].sort_values().index.tolist()
print("\nClusters ordered by increasing mean PC1 (K-Means):", centroid_order)

In [ ]:
# ============================================================
# 6.3 LINK PCA AXES TO PHYSIOLOGY
# ============================================================

# Loadings interpretation helper: show strongest contributors for a PC
def show_top_loadings(loadings_df, pc="PC1", n=7):
    print(f"\nTop + loadings for {pc}:")
    print(loadings_df[pc].sort_values(ascending=False).head(n))
    print(f"\nTop - loadings for {pc}:")
    print(loadings_df[pc].sort_values(ascending=True).head(n))

show_top_loadings(loadings, "PC1", n=5)
show_top_loadings(loadings, "PC2", n=5)

# Tip for report interpretation:
# - PC dominated by ALT/AST/GGT/BIL tends to reflect "injury/cholestasis/inflammation"
# - PC dominated by ALB/CHE/PROT/CHOL tends to reflect "synthetic/metabolic function"
# (Direction sign is arbitrary: + vs - does not change interpretation.)


In [ ]:
# ============================================================
# 6.4 OVERLAY POST HOC DIAGNOSTIC CATEGORIES
# ============================================================

plt.figure(figsize=(6, 5))
sc = plt.scatter(
    analysis_df["PC1"],
    analysis_df["PC2"],
    c=analysis_df["Category_num"],   # numeric codes for safe coloring
    s=10,
    alpha=0.7
)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA Projection Colored by Diagnostic Category (Post Hoc)")
plt.colorbar(sc, label="Category (numeric code)")
plt.tight_layout()
plt.show()

# OPTIONAL: If you prefer readable legend with labels (can be crowded):
def scatter_with_label_legend(df2d, title):
    plt.figure(figsize=(6, 5))
    for lab in sorted(df2d["Category_label"].unique()):
        mask = df2d["Category_label"] == lab
        plt.scatter(df2d.loc[mask, "PC1"], df2d.loc[mask, "PC2"], s=10, alpha=0.7, label=lab)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title(title)
    plt.legend(markerscale=2, bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

# Uncomment if you want:
# scatter_with_label_legend(analysis_df, "PCA Projection with Category Labels (Post Hoc)")


In [ ]:
# ============================================================
# 6.5 CATEGORY DISTRIBUTION PER CLUSTER
# ============================================================

# Use numeric categories for stable ordering in tables
cluster_category_table = pd.crosstab(
    analysis_df["Cluster_KMeans"],
    analysis_df["Category_num"],
    normalize="index"
)

print("\nCategory distribution within each K-Means cluster (row-normalized):")
print(cluster_category_table)

plt.figure(figsize=(8, 4))
plt.imshow(cluster_category_table.values, aspect="auto")
plt.xticks(range(len(cluster_category_table.columns)), cluster_category_table.columns)
plt.yticks(range(len(cluster_category_table.index)), cluster_category_table.index)
plt.xlabel("Category (numeric)")
plt.ylabel("Cluster (K-Means)")
plt.title("Cluster vs Category Distribution (Proportions)")
plt.colorbar(label="Proportion")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 6.6 MONOTONIC TREND ANALYSIS
# ============================================================

# Compute mean PC1 per diagnostic category (numeric)
category_means = analysis_df.groupby("Category_num")["PC1"].mean().sort_index()

print("\nMean PC1 per Category (numeric):")
print(category_means)

plt.figure()
plt.plot(category_means.index, category_means.values, marker="o")
plt.xlabel("Diagnostic Category (numeric)")
plt.ylabel("Mean PC1")
plt.title("Category Trend Along PC1 (Monotonicity Check)")
plt.tight_layout()
plt.show()

# OPTIONAL: also check PC2 trend
category_means_pc2 = analysis_df.groupby("Category_num")["PC2"].mean().sort_index()
plt.figure()
plt.plot(category_means_pc2.index, category_means_pc2.values, marker="o")
plt.xlabel("Diagnostic Category (numeric)")
plt.ylabel("Mean PC2")
plt.title("Category Trend Along PC2")
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# 6.7 OVERLAP ASSESSMENT (RANGES + BOX-PLOT)
# ============================================================

# Print PC1 min/max range by category to quantify overlap
for cat in sorted(analysis_df["Category_num"].unique()):
    subset = analysis_df[analysis_df["Category_num"] == cat]
    print(f"\nCategory {cat} — PC1 range:")
    print(f"Min: {subset['PC1'].min():.2f}, Max: {subset['PC1'].max():.2f}, "
          f"IQR: {(subset['PC1'].quantile(0.75) - subset['PC1'].quantile(0.25)):.2f}")

# Boxplot gives an immediate visual of overlap between adjacent categories
cats_sorted = sorted(analysis_df["Category_num"].unique())
data_for_box = [analysis_df.loc[analysis_df["Category_num"] == c, "PC1"].values for c in cats_sorted]

plt.figure(figsize=(8, 4))
plt.boxplot(data_for_box, labels=[str(c) for c in cats_sorted], showfliers=False)
plt.xlabel("Diagnostic Category (numeric)")
plt.ylabel("PC1")
plt.title("PC1 Distribution by Category (Overlap Assessment)")
plt.tight_layout()
plt.show()

# ============================================================
# OPTIONAL: "SEVERITY SCORE" IN LATENT SPACE (USEFUL FOR CONCLUSION)
# ============================================================
# A simple unsupervised severity proxy:
# - Use PC1 as a severity score if it shows monotonic trend with Category.
analysis_df["Severity_proxy_PC1"] = analysis_df["PC1"]

# Examine how clusters align with severity proxy
severity_by_cluster = analysis_df.groupby("Cluster_KMeans")["Severity_proxy_PC1"].mean().sort_values()
print("\nMean severity proxy (PC1) by K-Means cluster (sorted):")
print(severity_by_cluster)


## **Anomaly Detection and Atypical Profiles**

### 7.1 Statistical Outlier Detection

**Methods**

* Z-score
* IQR method

**Purpose**

* Identify global deviations from population norms

### 7.2 Model-based Anomaly Detection

**Methods**

* Isolation Forest
* Local Outlier Factor (LOF)

**Purpose**

* Detect local and structural anomalies
* Identify patients poorly represented by clusters


### Outputs

* Anomaly scores per patient
* Overlap between anomalies and advanced disease categories
* Identification of potential data quality issues


In [ ]:
# ============================================================
# 7) ANOMALY DETECTION & ATYPICAL PROFILES (FIXED)
# ============================================================

# ------------------------------------------------------------
# ASSUMPTION
# ------------------------------------------------------------
# Available:
# - X_scaled  → standardized feature matrix (DataFrame)
# - analysis_df → contains PCA columns (PC1, PC2, ...) + cluster labels
# - y_category → original Category series (strings like "0=Blood Donor" OR numeric)
# ------------------------------------------------------------

# Ensure X_scaled is a DataFrame with correct index
if isinstance(X_scaled, np.ndarray):
    X_scaled = pd.DataFrame(X_scaled, index=analysis_df.index)

# ============================================================
# 0) ENSURE CATEGORY COLUMNS EXIST IN analysis_df (NO KEYERROR)
# ============================================================

# If analysis_df already has Category_label/Category_num from Section 6, keep them.
# Otherwise rebuild them from y_category.
if ("Category_label" not in analysis_df.columns) or ("Category_num" not in analysis_df.columns):
    y_category_label = pd.Series(y_category, index=analysis_df.index).astype(str)

    # Extract leading digits from labels like "0=Blood Donor"
    y_category_num = pd.to_numeric(
        y_category_label.str.extract(r"^(\d+)")[0],
        errors="coerce"
    )

    # If extraction fails for some entries, map labels to codes
    if y_category_num.isna().any():
        label_map = {lab: i for i, lab in enumerate(sorted(y_category_label.unique()))}
        y_category_num = y_category_label.map(label_map).astype(int)
    else:
        y_category_num = y_category_num.astype(int)

    analysis_df["Category_label"] = y_category_label.values
    analysis_df["Category_num"] = y_category_num.values



In [ ]:
# ============================================================
# 7.1 STATISTICAL OUTLIER DETECTION
# ============================================================

# --- Z-score method (global outliers) ---
z_scores = np.abs(zscore(X_scaled, nan_policy="omit"))
z_outliers = (z_scores > 3).any(axis=1)

analysis_df["Z_Outlier"] = z_outliers.astype(int)
print("\nNumber of Z-score outliers:", int(z_outliers.sum()))

# --- IQR method (robust global outliers) ---
Q1 = X_scaled.quantile(0.25)
Q3 = X_scaled.quantile(0.75)
IQR = Q3 - Q1

iqr_outliers = ((X_scaled < (Q1 - 1.5 * IQR)) |
                (X_scaled > (Q3 + 1.5 * IQR))).any(axis=1)

analysis_df["IQR_Outlier"] = iqr_outliers.astype(int)
print("Number of IQR outliers:", int(iqr_outliers.sum()))



In [ ]:
# ============================================================
# 7.2 MODEL-BASED ANOMALY DETECTION
# ============================================================

# --- Isolation Forest (structural/global) ---
iso = IsolationForest(contamination=0.05, random_state=42)
iso.fit(X_scaled)

analysis_df["Isolation_Score"] = iso.decision_function(X_scaled)   # higher = normal
analysis_df["Isolation_Outlier"] = (iso.predict(X_scaled) == -1).astype(int)

print("\nIsolation Forest detected anomalies:", int(analysis_df["Isolation_Outlier"].sum()))

# --- LOF (local density anomalies) ---
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.05)
lof_pred = lof.fit_predict(X_scaled)

analysis_df["LOF_Score"] = lof.negative_outlier_factor_
analysis_df["LOF_Outlier"] = (lof_pred == -1).astype(int)

print("LOF detected anomalies:", int(analysis_df["LOF_Outlier"].sum()))


In [ ]:
# ============================================================
# 7.3 VISUALIZE ANOMALIES IN PCA SPACE
# ============================================================

plt.figure(figsize=(6, 5))
sc = plt.scatter(analysis_df["PC1"], analysis_df["PC2"],
                 c=analysis_df["Isolation_Outlier"], s=10, alpha=0.7)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Isolation Forest Outliers in PCA Space")
plt.colorbar(sc, label="Outlier (1=Yes)")
plt.tight_layout()
plt.show()


In [ ]:
# ============================================================
# 7.4 OVERLAP WITH DIAGNOSTIC CATEGORIES (POST HOC)
# ============================================================

iso_category_table = pd.crosstab(
    analysis_df["Isolation_Outlier"],
    analysis_df["Category_num"],
    normalize="columns"
)
print("\nIsolation Forest Outliers by Category (proportion within category):")
print(iso_category_table)

lof_category_table = pd.crosstab(
    analysis_df["LOF_Outlier"],
    analysis_df["Category_num"],
    normalize="columns"
)
print("\nLOF Outliers by Category (proportion within category):")
print(lof_category_table)


In [ ]:
# ============================================================
# 7.5 COMBINED ANOMALY FLAG
# ============================================================

analysis_df["Any_Anomaly"] = (
    analysis_df["Z_Outlier"] |
    analysis_df["IQR_Outlier"] |
    analysis_df["Isolation_Outlier"] |
    analysis_df["LOF_Outlier"]
).astype(int)

print("\nTotal patients flagged by any method:", int(analysis_df["Any_Anomaly"].sum()))


In [ ]:
# ============================================================
# 7.6 IDENTIFY EXTREME PROFILES
# ============================================================

most_anomalous = analysis_df.sort_values("Isolation_Score").head(10)

print("\nTop 10 most anomalous patients (Isolation Forest):")
print(most_anomalous[["Isolation_Score", "LOF_Score", "Category_label", "Category_num"]])

plt.figure(figsize=(6, 5))
plt.scatter(analysis_df["PC1"], analysis_df["PC2"], s=10, alpha=0.25)
plt.scatter(most_anomalous["PC1"], most_anomalous["PC2"], s=45, alpha=0.9)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Top 10 Anomalies Highlighted in PCA Space")
plt.tight_layout()
plt.show()

## **Post Hoc Clinical Interpretation**

### Use of Category variable

* Compare cluster membership with diagnostic categories
* Analyze distribution of categories per cluster
* Assess agreement without claiming causality

📌 *Important:* This step is interpretative, not evaluative.

In [ ]:
# ============================================================
# 8.1 Cluster vs Category Distribution (K-Means)
# ============================================================

# Cross-tabulation (counts)
cluster_vs_category_counts = pd.crosstab(
    analysis_df["Cluster_KMeans"],
    analysis_df["Category_label"]
)

print("\nCluster vs Category (Counts):")
print(cluster_vs_category_counts)

# Row-normalized proportions (within cluster)
cluster_vs_category_prop = pd.crosstab(
    analysis_df["Cluster_KMeans"],
    analysis_df["Category_label"],
    normalize="index"
)

print("\nCluster vs Category (Proportions within cluster):")
print(cluster_vs_category_prop)


In [ ]:
# ============================================================
# 8.2 Dominant Category per Cluster
# ============================================================

dominant_category = cluster_vs_category_prop.idxmax(axis=1)

print("\nDominant diagnostic category per cluster:")
print(dominant_category)


In [ ]:
# ============================================================
# 8.3 Cluster Purity (Descriptive, NOT predictive accuracy)
# ============================================================

# Purity = max proportion in each cluster
cluster_purity = cluster_vs_category_prop.max(axis=1)

print("\nCluster purity (max proportion per cluster):")
print(cluster_purity)

print("\nAverage cluster purity:",
      cluster_purity.mean())

In [ ]:
# ============================================================
# 8.4 Adjusted Rand Index (Optional, descriptive only)
# ============================================================

# This measures similarity between cluster assignment and diagnostic labels.
# IMPORTANT: This is descriptive similarity, NOT predictive accuracy.

from sklearn.metrics import adjusted_rand_score

ari = adjusted_rand_score(
    analysis_df["Category_num"],
    analysis_df["Cluster_KMeans"]
)

print("\nAdjusted Rand Index (Cluster vs Category):", ari)


## **Robustness and Sensitivity Analysis**

### Objectives

* Ensure findings are not artifacts of preprocessing or algorithm choice

### Methods

* Compare clustering across:

  * Different numbers of PCA components
  * Different algorithms (K-means vs GMM)
* Assess stability using silhouette score trends

In [ ]:
# ============================================================
# 9.1 Sensitivity to Number of PCA Components
# ============================================================

print("\n--- Sensitivity to PCA dimensionality ---")

pca_components_test = [2, 3, 4, 5, 6]
silhouette_results = []

for n_comp in pca_components_test:

    # Fit PCA with fixed number of components
    pca_test = PCA(n_components=n_comp, random_state=42)
    X_pca_test = pca_test.fit_transform(X_scaled)

    # K-Means clustering
    kmeans_test = KMeans(n_clusters=optimal_k, random_state=42, n_init=20)
    labels_test = kmeans_test.fit_predict(X_pca_test)

    sil = silhouette_score(X_pca_test, labels_test)
    silhouette_results.append(sil)

    print(f"PCA components: {n_comp}, Silhouette: {sil:.4f}")

# Plot stability
plt.figure()
plt.plot(pca_components_test, silhouette_results, marker="o")
plt.xlabel("Number of PCA Components")
plt.ylabel("Silhouette Score")
plt.title("Clustering Stability Across PCA Dimensions")
plt.show()


In [ ]:
# ============================================================
# 9.2 Compare Algorithms (K-Means vs GMM)
# ============================================================

print("\n--- Algorithm comparison (K-Means vs GMM) ---")

# Use your previously selected PCA representation
X_for_compare = X_pca_df.copy()

# K-Means
kmeans_labels = analysis_df["Cluster_KMeans"]

# GMM
gmm_labels = analysis_df["Cluster_GMM"]

# Agreement between clusterings
ari_between_algorithms = adjusted_rand_score(kmeans_labels, gmm_labels)

print("Adjusted Rand Index (KMeans vs GMM):", ari_between_algorithms)



In [ ]:
# ============================================================
# 9.3 Silhouette Comparison Across Algorithms
# ============================================================

sil_kmeans = silhouette_score(X_for_compare, kmeans_labels)
sil_gmm = silhouette_score(X_for_compare, gmm_labels)

print("\nSilhouette Scores:")
print("K-Means:", sil_kmeans)
print("GMM:", sil_gmm)


In [ ]:
# ============================================================
# 9.4 Interpretation Support Tables
# ============================================================

comparison_table = pd.DataFrame({
    "Method": ["K-Means", "GMM"],
    "Silhouette": [sil_kmeans, sil_gmm]
})

print("\nClustering Method Comparison:")
print(comparison_table)


## **10. Limitations and Methodological Constraints**


Explicitly acknowledge:

* Observational nature of data
* Overlap between disease stages
* Sensitivity to scaling and transformations
* Absence of longitudinal information

## **11. Conclusions – How Everything Comes Together**



Your conclusions should answer:

1. **Structure:**
   The data exhibits clear latent structure driven by liver inflammation and synthetic function markers.

2. **Stratification:**
   Unsupervised clustering reveals coherent patient subgroups, though boundaries are fuzzy and overlapping.

3. **Progression:**
   Disease severity appears as a **continuum rather than strictly discrete stages**.

4. **Anomalies:**
   Atypical profiles exist and often correspond to severe or complex cases not well captured by simple clustering.

5. **Value of unsupervised learning:**
   Unsupervised methods provide complementary insight to clinical classification by revealing heterogeneity and uncertainty.

